In [3]:
!export CUDA_VISIBLE_DEVICES=0

'export' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())


0


In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.device_count()

1

In [7]:
torch.cuda.current_device()

0

In [8]:
# cudaSetDevice(0)
#dataset transcript要在dataset資好夾調
#cuda 在basesolver init調
#dataset要在data.py設定

In [1]:
import torch
torch.cuda.is_available()
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
import torch
import argparse
import numpy as np
import pandas as pd

dfs = []
alldf =pd.DataFrame()

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/cv11Lu_asr_lstm4atthead_allvocab-whisperclass-test.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', None)
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',0)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', True)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 9)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
    force_cudnn_initialization()

    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            # from test_whisperclass import Solver
            from Whisper_Biclass_test import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            assert paras.load is not None
            from train_binaryclass2_5folds2 import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))
    for idx in range(5):
        paras.config = f'./config/cv11Lu_asr_lstm4atthead_allvocab-biclass-test-5-{idx+1}.yaml'
        config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)
        config['core']['ckpt'] = f'LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-{idx+1}_sd0/wav2vec-15k-r-a-final.pth'
        solver = Solver(config, paras, mode)

        solver.load_data()
    #     solver.print_model()
        solver.set_model()
        names, hyps, txts, ans = solver.exec()
        dfs.append(pd.DataFrame(
            {'name': names,
             'hyps': hyps,
             'truth': txts,
             'ifcorrect': ans
            }))
        print(dfs[idx])
        print(names)
        print(hyps)
        print(txts)
        print(ans)
        del solver


0


In [2]:
main()


Using test mode

[INFO] Evaluating result of tr. config @ ../LAS_Mandarin_PyTorch-master/config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1.yaml
[INFO] Evaluating result of tr. ckpt @ LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1_sd0/wav2vec-15k-r-a-final.pth


c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-1-16']
Mozillacv11Dataset CTT5-1-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 100
text len: 100
remove None, then wav data: 99, text len: 99
Setup ASR model and optimizer 
('data_process\\CTT5-1-16\\15CTT.wav',)   0.5295692086219788   0.0   False
('data_process\\CTT5-1-16\\16CTT.wav',)   0.6331841349601746   1.0   True
('data_process\\CTT5-1-16\\22CTT.wav',)   0.5417379140853882   0.0   False
('data_process\\CTT5-1-16\\28CTT.wav',)   0.6009120941162109   1.0   True
('data_process\\CTT5-1-16\\34CTT.wav',)   0.5316822528839111   0.0   False
('data_process\\CTT5-1-16\\35CTT.wav',)   0.5422372817993164   1.0  

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-2-16']
Mozillacv11Dataset CTT5-2-16 found wav data: 18
text len: 18
remove None, then wav data: 18, text len: 18
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 100
text len: 100
remove None, then wav data: 99, text len: 99
Setup ASR model and optimizer 
('data_process\\CTT5-2-16\\11CTT.wav',)   0.5866975784301758   1.0   True
('data_process\\CTT5-2-16\\25CTT.wav',)   0.43155378103256226   1.0   False
('data_process\\CTT5-2-16\\2CTT.wav',)   0.4161677360534668   0.0   True
('data_process\\CTT5-2-16\\37CTT.wav',)   0.4869794249534607   1.0   False
('data_process\\CTT5-2-16\\39CTT.wav',)   0.5239807963371277   1.0   True
('data_process\\CTT5-2-16\\45CTT.wav',)   0.4274613857269287   1.0   

...

('data_process\\CTT5-3-16\\87CTT.wav',)   0.5566044449806213   0.0   False
('data_process\\CTT5-3-16\\93CTT.wav',)   0.5048454999923706   0.0   False
('data_process\\CTT5-3-16\\98CTT.wav',)   0.4892975986003876   0.0   True
[INFO] All done !                                                                                          
                                name      hyps  truth  ifcorrect
0   data_process\CTT5-3-16\18CTT.wav  0.499541    1.0      False
1    data_process\CTT5-3-16\1CTT.wav  0.467757    0.0       True
2   data_process\CTT5-3-16\20CTT.wav  0.563907    1.0       True
3   data_process\CTT5-3-16\30CTT.wav  0.491529    0.0       True
4   data_process\CTT5-3-16\32CTT.wav  0.500124    1.0       True
5   data_process\CTT5-3-16\38CTT.wav  0.597213    1.0       True
6    data_process\CTT5-3-16\3CTT.wav  0.499897    0.0       True
7   data_process\CTT5-3-16\43CTT.wav  0.493836    0.0       True
8   data_process\CTT5-3-16\52CTT.wav  0.521219    0.0      False
9   data_process\C

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-4-16']
Mozillacv11Dataset CTT5-4-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 100
text len: 100
remove None, then wav data: 99, text len: 99
Setup ASR model and optimizer 
('data_process\\CTT5-4-16\\100CTT.wav',)   0.4967343807220459   0.0   True
('data_process\\CTT5-4-16\\12CTT.wav',)   0.5643101930618286   1.0   True
('data_process\\CTT5-4-16\\17CTT.wav',)   0.6593894958496094   1.0   True
('data_process\\CTT5-4-16\\21CTT.wav',)   0.4887368083000183   0.0   True
('data_process\\CTT5-4-16\\29CTT.wav',)   0.5266346335411072   0.0   False
('data_process\\CTT5-4-16\\4CTT.wav',)   0.5065240263938904   1.0   T

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-5-16']
Mozillacv11Dataset CTT5-5-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 100
text len: 100
remove None, then wav data: 99, text len: 99
Setup ASR model and optimizer 
('data_process\\CTT5-5-16\\10CTT.wav',)   0.600223958492279   1.0   True
('data_process\\CTT5-5-16\\13CTT.wav',)   0.518591046333313   0.0   False
('data_process\\CTT5-5-16\\14CTT.wav',)   0.4763298034667969   0.0   True
('data_process\\CTT5-5-16\\24CTT.wav',)   0.5320809483528137   1.0   True
('data_process\\CTT5-5-16\\31CTT.wav',)   0.5997482538223267   1.0   True
('data_process\\CTT5-5-16\\44CTT.wav',)   0.5340210199356079   0.0   Fal

In [3]:
for idx, df in enumerate(dfs):
    alldf = pd.concat([alldf, df])
    # df.to_csv(f'./biclass_result/whisper-WhisperForAudioClassification-v3-10k-{idx+1}_2', index=False)
alldf.to_csv(f'./biclass_result/wav2vec-15k-r-a-final.csv', index=False)
# whisper-v2-10k-{idx+1}_with_id_2.csv
# whisper-v2-15k_with_id_2.csv
# whisper-WhisperForAudioClassification-v3-10k-{idx+1}_2
# whisper-WhisperForAudioClassification-v3-10k_2.csv

In [ ]:
print('asd')

In [ ]:
!python eval.py --file ~/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv

Traceback (most recent call last):
  File "eval.py", line 33, in <module>
    result['hyp_char_cnt'] = result.apply(lambda x: len(x.hyp), axis=1)
  File "c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\pandas\core\frame.py", line 9423, in apply
    return op.apply().__finalize__(self, method="apply")
  File "c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\pandas\core\apply.py", line 678, in apply
    return self.apply_standard()
  File "c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\pandas\core\apply.py", line 798, in apply_standard
    results, res_index = self.apply_series_generator()
  File "c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\pandas\core\apply.py", line 814, in apply_series_generator
    results[i] = self.f(v)
  File "eval.py", line 33, in <lambda>
    result['hyp_char_cnt'] = result.apply(lambda x: len(x.hyp), axis=1)
  File "c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\pandas\core\generic.py", line 5989, in __getattr__
    return

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
import numpy as np
y_true = [0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0] 
y_pred = [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
target_names = ['non-patient', 'patient']
print(classification_report(y_true, y_pred, target_names=target_names))
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print('sensitivity: ',tp / (fn+tp))
print('specificity: ',tn / (fp+tn))
pred = np.array([0.0443, 0.0140, 0.3337, 0.0560, 0.0385, 0.5118, 0.1961, 0.1746, 0.0268, 0.2939, 0.0169, 0.6525, 0.3661, 0.1010, 0.0077, 0.0994, 0.2264, 0.6143, 0.0750, 0.4048, 0.2368])
fpr, tpr, thresholds = metrics.roc_curve(np.array(y_true), np.array(pred), pos_label=1)
print('AUC: ',metrics.auc(fpr, tpr))

              precision    recall  f1-score   support

 non-patient       0.80      0.92      0.86        13
     patient       0.83      0.62      0.71         8

    accuracy                           0.81        21
   macro avg       0.82      0.77      0.79        21
weighted avg       0.81      0.81      0.80        21

sensitivity:  0.625
specificity:  0.9230769230769231
AUC:  0.5288461538461539
